In [ ]:
import pybedtools
import pandas as pd
import numpy as np
from pybedtools import BedTool

In [ ]:
# this code replicates Figure 3d in the SHARPR paper

# the following code is meant to help you implement your own version
# and will not work unless you make some modifications

from pybedtools import BedTool

predicted = pd.read_pickle('/u/home/m/mudiyang/scratch/test/SHARPR.pkl')

df = predicted

coord_df = pd.read_csv('/u/home/m/mardren/scratch/SHARPR/coords.tsv', 
                           sep='\t', index_col=0)

coord_df.columns = ['chrom','start','stop']

absmax_idx = df.abs().idxmax(axis=1)
# sym_idx = (294-absmax_idx.map(int)).map(str)
# center_idx = pd.Series(147, index=df.index).map(str)

idx_d = {"sharpr_minP_model": absmax_idx}

for current_idx, idx in idx_d.items():
    coord_df['absmax_idx'] = idx
    coord_df['absmax_pos'] = coord_df['start'].map(int) + coord_df['absmax_idx'].map(int)

    absmax_pos_df = pd.concat(
        [
            coord_df["chrom"],
            coord_df["absmax_pos"],
            (coord_df["absmax_pos"] + 1).rename("absmax_pos_"),
            coord_df.index.to_series(),
        ],
        axis=1,
    )

    sharpr_absmax = BedTool.from_dataframe(absmax_pos_df).sort()

    absmax_values = pd.Series(df.lookup(df.index, idx), index=df.index, name='absmax_values')

    centipede = BedTool('/u/project/ernst/dincer/tiling_2020/data/processed/centipede/centipede_HepG2.hg19.bed')

    hits = sharpr_absmax.intersect(centipede, c=True)
    hits_df = hits.to_dataframe().set_index('name').rename_axis('region_id')

    hits_df['absmax_values'] = absmax_values
    hits_df['abs_absmax_values'] = absmax_values.abs()

    hits_df = hits_df.sort_values('abs_absmax_values', ascending=False)

    hits_df['is_centipede'] = hits_df['score'] > 0

    hits_df['i'] = hits_df.reset_index().reset_index().set_index('region_id')['index']+1

    hits_df['centipede_cumsum'] = hits_df['is_centipede'].cumsum()

    hits_df[f'centipede_motif_overlap'] = (hits_df['centipede_cumsum']/hits_df['i'])
    
#     hits_df = hits_df[hits_df['i'] > 200]
    
    absmax_df = hits_df[['i','centipede_motif_overlap']]

In [ ]:
# this code replicates Figure 3d in the SHARPR paper

# the following code is meant to help you implement your own version
# and will not work unless you make some modifications

from pybedtools import BedTool

SHARPR_Paper = pd.read_csv('/u/home/m/mudiyang/scratch/SHARPR_Scores/HEPG2_SHARPR-MPRA_scores/basepredictions_HEPG2_ScaleUpDesign1and2_SV40P.txt',
                    sep='\t', index_col=0, header = None)

SHARPR_Paper.columns = range(0,295)


df = SHARPR_Paper

coord_df = pd.read_csv('/u/home/m/mardren/scratch/SHARPR/coords.tsv', 
                           sep='\t', index_col=0)

coord_df.columns = ['chrom','start','stop']

absmax_idx = df.abs().idxmax(axis=1)
# sym_idx = (294-absmax_idx.map(int)).map(str)
# center_idx = pd.Series(147, index=df.index).map(str)

idx_d = {"sharpr_minP_model": absmax_idx}

for current_idx, idx in idx_d.items():
    coord_df['absmax_idx'] = idx
    coord_df['absmax_pos'] = coord_df['start'].map(int) + coord_df['absmax_idx'].map(int)

    absmax_pos_df = pd.concat(
        [
            coord_df["chrom"],
            coord_df["absmax_pos"],
            (coord_df["absmax_pos"] + 1).rename("absmax_pos_"),
            coord_df.index.to_series(),
        ],
        axis=1,
    )

    sharpr_absmax = BedTool.from_dataframe(absmax_pos_df).sort()

    absmax_values = pd.Series(df.lookup(df.index, idx), index=df.index, name='absmax_values')

    centipede = BedTool('/u/project/ernst/dincer/tiling_2020/data/processed/centipede/centipede_HepG2.hg19.bed')

    hits = sharpr_absmax.intersect(centipede, c=True)
    hits_df = hits.to_dataframe().set_index('name').rename_axis('region_id')

    hits_df['absmax_values'] = absmax_values
    hits_df['abs_absmax_values'] = absmax_values.abs()

    hits_df = hits_df.sort_values('abs_absmax_values', ascending=False)

    hits_df['is_centipede'] = hits_df['score'] > 0

    hits_df['i'] = hits_df.reset_index().reset_index().set_index('region_id')['index']+1

    hits_df['centipede_cumsum'] = hits_df['is_centipede'].cumsum()

    hits_df[f'centipede_motif_overlap'] = (hits_df['centipede_cumsum']/hits_df['i'])
    
#     hits_df = hits_df[hits_df['i'] > 200]
    
    Orig_df = hits_df[['i','centipede_motif_overlap']]

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(8,5), dpi=250)
plt.plot(
        hits_df['i'],
        absmax_df['centipede_motif_overlap'],'b', label = 'MaxPos (position of maximum absolute score)'
    )
plt.plot(
        hits_df['i'],
        Orig_df['centipede_motif_overlap'], 'y', label = 'Original Sharpr'
    )
plt.legend()
plt.title("Megan's Model: SHARPR Scores for Sequence Predicted Values CENTIPEDE Overlap")
plt.xlabel("Region rank based on HepG2 Sharpr-MPRA score at MaxPos/ CenPos/SymPos position")
plt.ylabel("HepG2 CENTIPEDE motif overlap at MaxPos/CenPos/SymPos nucleotide")
plt.show()